## 1. Classic American names
<p>How have American baby name tastes changed since 1920?  Which names have remained popular for over 100 years, and how do those names compare to more recent top baby names? </p>
<p>This data is provided by the United States Social Security Administration, which lists first names along with the number and sex of babies they were given to in each year. For processing speed purposes, the dataset have been limited to first names which were given to over 5,000 American babies in a given year. This data spans 101 years, from 1920 through 2020.</p>
<h3 id="baby_names"><code>baby_names</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>year</td>
</tr>
<tr>
<td style="text-align:left;"><code>first_name</code></td>
<td>varchar</td>
<td>first name</td>
</tr>
<tr>
<td style="text-align:left;"><code>sex</code></td>
<td>varchar</td>
<td><code>sex</code> of babies given <code>first_name</code></td>
</tr>
<tr>
<td style="text-align:left;"><code>num</code></td>
<td>int</td>
<td>number of babies of <code>sex</code> given <code>first_name</code> in that <code>year</code></td>
</tr>
</tbody>
</table>

In [ ]:
%%sql
postgresql:///names
    
-- Get the total number of children with each name
SELECT first_name, SUM(num) AS sum
FROM baby_names
GROUP BY first_name
HAVING COUNT(DISTINCT year) = 101
ORDER BY sum DESC;

## 2. Timeless or trendy?
<p>Let's broaden our understanding of the dataset by looking at all names. Was the name classic and popular across many years or trendy, only popular for a few years?</p>

In [ ]:
%%sql

SELECT first_name, SUM(num) AS sum,
CASE WHEN COUNT(year) > 80 THEN 'Classic'
        WHEN COUNT(year) > 50 THEN 'Semi-classic'
        WHEN COUNT(year) > 20 THEN 'Semi-trendy'
        ELSE 'Trendy' END AS popularity_type
FROM baby_names
GROUP BY first_name
ORDER BY first_name;

## 3. Top-ranked female names since 1920
<p>What are the top-ranked female names since 1920?</p>

In [ ]:
%%sql

SELECT RANK() OVER(ORDER BY SUM(num) DESC) AS name_rank, first_name, SUM(num) AS sum
FROM baby_names
WHERE sex = 'F'
GROUP BY first_name
LIMIT 10;

## 4. Picking a baby name
<p>Let's find traditionally female name ending in the letter 'a' and a name that has been popular in the years since 2015. </p>

In [ ]:
%%sql

SELECT first_name 
FROM baby_names
WHERE sex = 'F' AND year > 2015 AND first_name LIKE '%a'
GROUP BY first_name
ORDER BY SUM(num) DESC;

## 5. The popular female name expansion
<p>When did the most popular female name become so popular?</p>

In [ ]:
%%sql

SELECT year, first_name, num, SUM(num) OVER (ORDER BY year) AS cumulative_number
FROM baby_names
WHERE first_name = (
    SELECT first_name 
    FROM baby_names
    WHERE sex = 'F' AND year > 2015 AND first_name LIKE '%a'
    GROUP BY first_name
    ORDER BY SUM(num) DESC
    LIMIT 1)
ORDER BY year;

## 6. Top male names over the years

In [ ]:
%%sql

SELECT b.year, b.first_name, b.num
FROM baby_names AS b
INNER JOIN (
    SELECT year, MAX(num) as max_num
    FROM baby_names
    WHERE sex = 'M'
    GROUP BY year) AS subquery 
ON subquery.year = b.year 
    AND subquery.max_num = b.num
ORDER BY year DESC;

## 7. The most years at number one
<p>Which name has been number one for the largest number of years?</p>

In [ ]:
%%sql

WITH top_male_names AS (SELECT b.year, b.first_name, b.num
FROM baby_names AS b
INNER JOIN (
    SELECT year, MAX(num) as max_num
    FROM baby_names
    WHERE sex = 'M'
    GROUP BY year) AS subquery 
ON subquery.year = b.year 
    AND subquery.max_num = b.num
ORDER BY year DESC)

SELECT first_name, COUNT(first_name) AS count_top_name
FROM top_male_names
GROUP BY first_name
ORDER BY count_top_name DESC;